In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [31]:
from os.path import join as path
from torchvision.transforms import (
    Compose, Resize, Normalize, ToPILImage,
    ToTensor, RandomHorizontalFlip)
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader


tform = {'train': Compose([
                           Resize((128, 128)),
                           ToTensor(),
                           RandomHorizontalFlip(),
                           Normalize((0.485, 0.456, 0.406),
                                     (0.229, 0.224, 0.225))]),
         'valid': Compose([
                           Resize((128, 128)),
                           ToTensor(),
                           Normalize((0.485, 0.456, 0.406),
                                     (0.229, 0.224, 0.225))]),
         'test': Compose([
                           Resize((128, 128)),
                           ToTensor(),
                           Normalize((0.485, 0.456, 0.406),
                                     (0.229, 0.224, 0.225))])}

data_dir = '/content/drive/MyDrive/Colab Notebooks/dogImages'
dataset = {x: ImageFolder(path(data_dir, x), tform[x]) for x in tform.keys()}
loaders = {x: DataLoader(dataset=dataset[x],
                         batch_size=64,
                         shuffle=(x=='train'),
                         num_workers=4,
                         pin_memory=True,
                         drop_last=True) for x in tform.keys()}

In [ ]:
from torch.nn import Module, Conv2d, MaxPool2d, Linear
from torch.nn.functional import relu, softmax, dropout
from torch import cuda, device

#TODO: Model
class Model(Module):
    def __init__(self):
        super(Model, self).__init__()
    
    def forward(self, x):
        return x

model = Model()
print(model)

use_cuda = cuda.is_available()
if use_cuda:
    model = model.cuda()

In [34]:
from torch.nn import CrossEntropyLoss
from torch.optim import Adam

criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=1e-3)

In [ ]:
def train(model, loaders, criterion, optimizer, epochs):
    for epoch in range(epochs):
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()
            elif phase == 'valid':
                model.eval()

            for data, cls in loaders[phase]:
                if use_cuda:
                    data = data.cuda()
                    cls = cls.cuda()

                if phase == 'train':
                    oprimizer.zero_grad()
                
                pred = model(data)
                loss = criterion(pred, cls)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

def test(model, loaders, criterion):
    model.eval()
    for data, cls in loaders['test']:
        if use_cuda:
            data = data.cuda()
            cls = cls.cuda()

        pred = model(data)
        loss = criterion(pred, cls)